<a href="https://colab.research.google.com/github/jgiornazi/test2/blob/master/CSCI3202_FinalProject_ConnectFour_Giornazi_Jamal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

**By writing your name below, you agree to abide by these rules:**

**Your name:** Jamal Giornazi

---


---

Some useful packages and libraries:



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [ ]:
class State:
    def __init__(self, moves):
        self.to_move = 'R'
        self.utility = 0
        self.board = {}
        self.moves = moves
    
    def __lt__(self, other):
      # Compare the utility of the current state with the other state
      return self.utility < other.utility

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        
        # your code goes here...
        # at current state apply move to the board and change the state
        h = cp.deepcopy(state)
        legal = [item for item in h.moves if item[1] == move[1]]

        for item in legal:
          if item[0] < move[0]:
            move = item

        h.board[move] = h.to_move
        
        h.utility = self.compute_utility(move,h)
        h.moves.remove(move)
        h.to_move = ('B' if state.to_move == 'R' else 'R')
        return h

    def astar_result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        
        # your code goes here...
        # at current state apply move to the board and change the state
        h = cp.deepcopy(state)
        legal = [item for item in h.moves if item[1] == move[1]]

        for item in legal:
          if item[0] < move[0]:
            move = item

        h.board[move] = h.to_move
        
        h.utility = self.a_star_huristic(move,h)
        h.moves.remove(move)
        h.to_move = ('B' if state.to_move == 'R' else 'R')
        return h

    def a_star_huristic(self, move, state):
      # your code goes here...
      board = cp.deepcopy(state.board)
      
      h = cp.deepcopy(state)
      legal = [item for item in h.moves if item[1] == move[1]]
      for item in legal:
        if item[0] < move[0]:
          move = item
      board[move] = state.to_move
      row, col = move
      player = state.to_move #r or b
      opp = ('B' if state.to_move == 'R' else 'R') 
      wincon = self.nwin
      ncol = self.ncol
      nrow = self.nrow

      #this works by == if true 1 else -1 and adding it makes it hit 3
      # print("the move is " + str(move) + "the player is" + str(player))
      in_a_row = 0
      for column in range(1, ncol+1):
        if(board.get((row, column)) == player):
          in_a_row += 1
          if(in_a_row == wincon):
            break
        else:
          in_a_row = 0 

      in_a_col = 0
      for r in range(1,nrow+1):
          if board.get((r,col))==player:
            in_a_col += 1
            if(in_a_col == wincon):
              break
          else:
            in_a_col = 0

      in_a_diag1 = 0
      for r in range(row,0,-1):
          if board.get((r,col-(row-r)))==player:
            in_a_diag1 += 1
            if in_a_diag1 == wincon:
              break
          else:
            in_a_diag1 = 0

          
      for r in range(row+1,nrow+1):
          if in_a_diag1 == wincon:
            break
          if board.get((r,col-(row-r)))==player:
            in_a_diag1 += 1
            if in_a_diag1 == wincon:
              break
          else:
            in_a_diag1 = 0

      
      in_a_diag2 = 0
      for r in range(row,0,-1):
          if board.get((r,col+(row-r)))==player:
            in_a_diag2 += 1
            if in_a_diag2 == wincon:
              break
          else:
            in_a_diag2 = 0
            
          
      for r in range(row+1,nrow+1):
          if in_a_diag2 == wincon:
            break
          if board.get((r,col+(row-r)))==player:
            in_a_diag2 += 1
            if in_a_diag2 == wincon:
              break
          elif board.get((r,col+(row-r))) == opp:
            in_a_diag2 = 0
          else:
            in_a_diag2 = 0
      value = 0
      thelist = [in_a_row, in_a_col, in_a_diag1, in_a_diag2]
      value = 0.1*thelist.count(1) + 0.3*thelist.count(2) + 0.9*thelist.count(3) + 1000*thelist.count(wincon)
      return value
    def path_cost(self,state, player):
      cost = 0
      board = cp.deepcopy(state.board)
      
      for p1 in board.values():
        if(p1 == player):
          cost+=1

      return cost
    def astar_game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        
        # your code goes here... 
        #if game is over ie the value is 100 cuz others cant be  
        a = state.utility
        return abs(a)>=1000 or len(state.moves)==0   
 


    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        '''        
        
        # your code goes here...
        board = cp.deepcopy(state.board)
        
        h = cp.deepcopy(state)
        legal = [item for item in h.moves if item[1] == move[1]]
        for item in legal:
          if item[0] < move[0]:
            move = item
        board[move] = state.to_move
        row, col = move
        player = state.to_move #r or b
        opp = ('B' if state.to_move == 'R' else 'R') 
        wincon = self.nwin
        ncol = self.ncol
        nrow = self.nrow

        #this works by == if true 1 else -1 and adding it makes it hit 3
        # print("the move is " + str(move) + "the player is" + str(player))
        in_a_row = 0
        for column in range(1, ncol+1):
          if(board.get((row, column)) == player):
            in_a_row += 1
            if(in_a_row == wincon):
              break
          else:
            in_a_row = 0 

        in_a_col = 0
        for r in range(1,nrow+1):
            if board.get((r,col))==player:
              in_a_col += 1
              if(in_a_col == wincon):
                break
            else:
              in_a_col = 0
      
        in_a_diag1 = 0
        for r in range(row,0,-1):
            if board.get((r,col-(row-r)))==player:
              in_a_diag1 += 1
              if in_a_diag1 == wincon:
                break
            else:
              in_a_diag1 = 0

            
        for r in range(row+1,nrow+1):
            if in_a_diag1 == wincon:
              break
            if board.get((r,col-(row-r)))==player:
              in_a_diag1 += 1
              if in_a_diag1 == wincon:
                break
            else:
              in_a_diag1 = 0

        
        in_a_diag2 = 0
        for r in range(row,0,-1):
            if board.get((r,col+(row-r)))==player:
              in_a_diag2 += 1
              if in_a_diag2 == wincon:
                break
            else:
              in_a_diag2 = 0
              
            
        for r in range(row+1,nrow+1):
            if in_a_diag2 == wincon:
              break
            if board.get((r,col+(row-r)))==player:
              in_a_diag2 += 1
              if in_a_diag2 == wincon:
                break
            elif board.get((r,col+(row-r))) == opp:
              in_a_diag2 = 0
            else:
              in_a_diag2 = 0

        # print(move, player, in_a_row, in_a_col, in_a_diag1, in_a_diag2)
        if wincon in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='R' else -1
        else:
            return 0 

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        
        # your code goes here...   
        return state.utility!=0 or len(state.moves)==0   

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        ''' A* star does it a bit different '''
        # your code goes here...
        return state.utility if player=='R' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of Connect Four!'''
        
        # your code goes here...
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    # self.display()
                    #commented out cuz it was soo annoying
                    return self.state.utility  
        
                

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [ ]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    # your code goes here...
    possible_actions = game.state.moves
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]
    


We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [ ]:
# rwins =0
# conntect = ConnectFour(10,10,6)
# out = conntect.play_game(random_player, random_player)
# out

1

In [ ]:
# 1000 games between two random players
winp = []
rwins=0
dwins=0
bwins=0
for i in range(1000):
  conntect = ConnectFour()
  out = conntect.play_game(random_player, random_player)
  if (out == 1):
    rwins += 1
  if (out == 0):
    dwins += 1
  if (out == -1):
    bwins += 1
  out
winp.append(rwins)
winp.append(dwins)
winp.append(bwins)
winp.append(rwins/1000)
print(winp)


# Your code here

[544, 2, 454, 0.544]


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [ ]:
# 1000 games between two random players

# Your code here

winp = []

rwins = 0
dwins = 0 
bwins = 0
for i in range(1000):
  conntect = ConnectFour(nrow=10,ncol=10, nwin = 6)
  out = conntect.play_game(random_player, random_player)
  if (out == 1):
    rwins += 1
  if (out == 0):
    dwins += 1
  if (out == -1):
    bwins += 1
  out
winp.append(rwins)
winp.append(dwins)
winp.append(bwins)
winp.append(rwins/1000)
print(winp)
print(dwins/1000)


[476, 93, 431, 0.476]
0.093


This is probably because there are way more options for a random player to place their chip and so it near impossible to get 4 in a row where the state space is so massive. There are probably substationally more statespaces as draws then wins as well because imagine trying to get 4 in a row on a massive board randomly, it would be like 1/x * 1/(x-1)... which is an incrdiably small number (and x depends on the size of the state space/board) 

I had one where it drew every single time which i thought is the answer as four in a row rnadomly is very diffcult. SO im not sure what happened but overall more draws happened

Win rate decressed by 10% on average and draws went up. More draws happened as it went from 0 draws to .101 or 10%


Again overall I want to say that way more draws (2 ->93) and less red wins bceause again more places (States) for a random player to place a chip and less liekly to get 4 in a row

### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [ ]:
# def alphabeta_search(game, max_player):
#   state = game.state
#   alpha = float("-inf")
#   beta = float("inf")
#   value = max_value(game, state, alpha, beta)
#   best_action = game.state.moves[0]
#   for action in state.moves:
#     if(game.result(action, state).utility >= value):
#       best_action = action
#       break
#   return best_action

# def max_value(game, state,alpha,beta):
#   if state.utility == 1 or state.utility == -1:
#     return state.utility
#   value = float("-inf")
#   for action in state.moves:
#     value = max(value, min_value(game, game.result(action, state), alpha, beta))
#     if value >= beta: return value
#     alpha = max(value, alpha)
#   return value

# def min_value(game, state, alpha, beta):
#   if state.utility == 1 or state.utility == -1:
#     return state.utility
#   value = float("inf")
#   for action in state.moves:
#     value = min(value, max_value(game, game.result(action,state), alpha, beta))
#     if value <= alpha: return value
#     beta = min(value, beta)
#   return value


In [ ]:
#https://www.freecodecamp.org/news/minimax-algorithm-guide-how-to-create-an-unbeatable-ai/ from CA Office hours
#alosed used psehdo code from class


def alpha_beta_search(game):
    state = game.state
    player = state.to_move
    # Body of alpha_beta_search:
    best_score = -np.inf
    alpha = -np.inf
    beta = np.inf
    best_action = None
    states_expanded = 0

    def red_max_value(game,state,alpha,beta, player):
      nonlocal states_expanded
      states_expanded += 1
      p1 = player
      if(game.game_over(state)):
        return game.utility(state, p1)
      v = -np.inf
      for a in state.moves:
        v = max(v, red_min_value(game, game.result(a, state), alpha, beta, p1))
        if v >= beta:
          return v
        alpha = max(v,alpha)
      return v


    def red_min_value(game,state,alpha,beta, player):
      nonlocal states_expanded
      states_expanded += 1
      p1 = player
      if(game.game_over(state)):
        return game.utility(state, p1)
      v = np.inf
      for a in state.moves:
        v = min(v, red_max_value(game, game.result(a, state), alpha, beta,p1))
        if v <= alpha:
          return v
        beta = min(v,beta)
      return v

    for a in state.moves:
        v = red_min_value(game, game.result(a, state), alpha, beta,player)
        if v > best_score:
            best_score = v
            alpha = v
            best_action = a
    print(states_expanded)
    return best_action

In [ ]:
#https://www.freecodecamp.org/news/minimax-algorithm-guide-how-to-create-an-unbeatable-ai/ from CA Office hours
#alosed used psehdo code from class


def minMax(game):
    state = game.state
    player = state.to_move
    # Body of alpha_beta_search:
    best_score = -np.inf
    alpha = -np.inf
    beta = np.inf
    best_action = None
    states_expanded = 0

    def red_max_value(game,state,alpha,beta, player):
      nonlocal states_expanded
      states_expanded += 1
      p1 = player
      if(game.game_over(state)):
        return game.utility(state, p1)
      v = -np.inf
      for a in state.moves:
        v = max(v, red_min_value(game, game.result(a, state), alpha, beta, p1))
        if alpha >= beta:
          return v
        alpha = max(v,alpha)
      return v


    def red_min_value(game,state,alpha,beta, player):
      nonlocal states_expanded
      states_expanded += 1
      p1 = player
      if(game.game_over(state)):
        return game.utility(state, p1)
      v = np.inf
      for a in state.moves:
        v = min(v, red_max_value(game, game.result(a, state), alpha, beta,p1))
        if beta <= alpha:
          return v
        beta = min(v,beta)
      return v

    for a in state.moves:
        v = red_min_value(game, game.result(a, state), alpha, beta,player)
        if v > best_score:
            best_score = v
            alpha = v
            best_action = a
    print(states_expanded)
    return best_action

In [ ]:
def minmax_player(game):
  action = alpha_beta_search(game)
  return action


def minmax_player_no(game):
  action = minMax(game)
  return action


Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [ ]:
#red always wins
winp = []
rwins= 0
dwins = 0
bwins = 0
for i in range(10):
  conntect = ConnectFour(nrow=3, ncol=4, nwin=3)
  out = conntect.play_game(minmax_player, random_player)
  if (out == 1):
    rwins += 1
  if (out == 0):
    dwins += 1
  if (out == -1):
    bwins += 1
  out
winp.append(rwins)
winp.append(dwins)
winp.append(bwins)
print(winp)

163118
8110
1084
94
R R B B 
R . B . 
R . . . 
163118
18706
1651
134
21
3
R B B R 
B R R B 
R B R . 
163118
18737
1521
146
21
2
R B B R 
B R R B 
R R . B 
163118
8110
349
R R R B 
. . . B 
. . . . 
163118
18706
2752
134
8
R B B R 
B R B . 
R . R . 
163118
18737
1661
64
R B B R 
. B R . 
. R . . 
163118
18706
1651
209
33
2
R R B R 
B R R B 
B R . B 
163118
8110
559
R R R B 
B . . . 
. . . . 
163118
8110
349
R R R B 
. . . B 
. . . . 
163118
8110
1084
97
R R B B 
R B . . 
R . . . 
[10, 0, 0]


In [ ]:
#red always wins
#im only running this once cuz im using this for the pruning question
#gonna submiut an interupte and gonna wait at most 15 min

# winp = []
# rwins= 0
# dwins = 0
# bwins = 0
# for i in range(1):
#   conntect = ConnectFour(nrow=3, ncol=4, nwin=3)
#   out = conntect.play_game(minmax_player_no, random_player)
#   if (out == 1):
#     rwins += 1
#   if (out == 0):
#     dwins += 1
#   if (out == -1):
#     bwins += 1
#   out
# winp.append(rwins)
# winp.append(dwins)
# winp.append(bwins)
# print(winp)

KeyboardInterrupt: ignored

In [ ]:
#the issue with mine is that the first players goal is to minimize the other 
# persons chance of winning
#but when black plays its not stopping the red from winning 
#TODO is to make it so the algo is flipped for black and red (ie since black is 
#-1 we want to make it positive one while red is -1)
#best way to do that is to flip copy and paste min max functions 
#throw a neg on the return
#and bam should result in black winnning against red


#the above idea didnt work to cauze draws: im getting same board over and over

#im thinking that its not catching losesing which is weird cuz it won every 
#Match against the random (I think im really screwing up the min max)
winp = []
rwins = 0
dwins=0
bwins =0 
for i in range(10):
  conntect = ConnectFour(nrow=3, ncol=4, nwin=3)
  out = conntect.play_game(minmax_player, minmax_player)
  if (out == 1):
    rwins += 1
  if (out == 0):
    dwins += 1
  if (out == -1):
    bwins += 1
  out
winp.append(rwins)
winp.append(dwins)
winp.append(bwins)
print(winp)

163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
163118
90735
18737
7475
1054
358
72
R B B R 
R B . . 
R . . . 
[10, 0, 0]


In [ ]:
winp = []
rwins= 0
dwins=0
bwins=0
for i in range(10):
  conntect = ConnectFour(nrow=3, ncol=4, nwin=3)
  out = conntect.play_game(random_player, minmax_player)
  if (out == 1):
    rwins += 1
  if (out == 0):
    dwins += 1
  if (out == -1):
    bwins += 1
  out
winp.append(rwins)
winp.append(dwins)
winp.append(bwins)
print(winp)

146330
4097
438
60
B B R R 
R B . R 
. B . . 
104745
4919
719
40
B R B R 
B . R R 
B . . . 


KeyboardInterrupt: ignored

### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

The code above was implimented to do some calculations 

Pruninig was done with a nodes of totall 99176 if i use v< alpha or v >beta

but for current maximizing of alpha i have it at 163118 

Honsetly not sure but I dont know overally it should cut by about 50% which this does I waited 20 min for a normal min max without an alpha beta or v check but it wasnt running so I gave up on it

In [ ]:
99176/163118 

0.6080015694160056

### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

 Value = My material - Opponent's material, where the material is a count of all the contiguous lines of pieces that a player has, weighted thus: 0.1 points for 1-in-a-row, 0.3 for 2-in-a-row and 0.9 for 3-in-a-row.


 This is taken from http://www.cs.cornell.edu/boom/2001sp/Anvari/Anvari.htm


In [ ]:
import queue
import heapq


In [ ]:
# start = 1
# end = 2
# queue = []
# heapq.heapify(queue)
# heapq.heappush(queue, start)
# heapq.heappush(queue, end)
# print(queue)
# state = heapq.heappop(queue)

# print(state)

[1, 2]
1


In [ ]:
def a_star_search(game):
  start = game.state
  player = start.to_move
  queue = []
  heapq._heapify_max(queue)
  heapq.heappush(queue, start)
  visited = []
  while len(queue) > 0:
    state = heapq._heappop_max(queue)
    if game.astar_game_over(state):
      if(player == "R" and game.utility(state,player) > 0):
        action = best_action(state, start)
        return action
      elif(player == "B" and game.utility(state,player)  < 0):
        action = best_action(state, start)
        return action
      else:
        continue
    if state.board in visited:
      continue
    visited.append(state.board)
    for moves in state.moves:
      new_state = game.astar_result(moves, state)
      heapq.heappush(queue, new_state)
      
  return random_player(game) #i think it was stuck at the begining of the 
  #game cuz it wouldnt play one cuz there was no move to compare i think


def best_action(state, start):
  end_board= state.board
  start_board = start.board
  end_board = set(end_board)
  start_board = set(start_board)
  a= end_board ^ start_board
  best = list(a)[0]

  return best



### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

In [ ]:
# Your code here.
winp = []
rwins= 0
dwins=0
bwins=0
for i in range(10):
  conntect = ConnectFour(nrow=6, ncol=7, nwin=4)
  out = conntect.play_game(random_player, a_star_search )
  if (out == 1):
    rwins += 1
  if (out == 0):
    dwins += 1
  if (out == -1):
    bwins += 1
  out
winp.append(rwins)
winp.append(dwins)
winp.append(bwins)
print(winp)

R B B R . R B 
R . R B . R B 
B . R R . . B 
R . . B . . B 
. . . . . . . 
. . . . . . . 
B B R R B B R 
R R B B R R B 
B B . R . R R 
. . . R . . R 
. . . . . . B 
. . . . . . B 
B B R B R R B 
. . R B . R B 
. . R B . . R 
. . . B . . R 
. . . . . . . 
. . . . . . . 
B R . R B R R 
R . . B R . B 
R . . B R . B 
. . . . . . B 
. . . . . . B 
. . . . . . . 
R R B B B R B 
B . R B R B R 
R . . R B R R 
B . . R . B B 
. . . R . . . 
. . . B . . . 
R R B B R R B 
B R R B B R R 
R R B B R . B 
. B R R B . B 
. R . . . . B 
. . . . . . B 
R R R R . . B 
B . . B . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
R . R R R R B 
B . . . . R B 
B . . . . . . 
B . . . . . . 
. . . . . . . 
. . . . . . . 
R B R B B . R 
. R . B R . B 
. R . B R . . 
. . . B . . . 
. . . . . . . 
. . . . . . . 
. R R B R R B 
. . B B B B R 
. . R . R . B 
. . . . . . . 
. . . . . . . 
. . . . . . . 
[3, 0, 7]


In [ ]:
# Your code here.
winp = []
rwins= 0
dwins=0
bwins=0
for i in range(10):
  conntect = ConnectFour(nrow=3, ncol=4, nwin=3)
  out = conntect.play_game(minmax_player, a_star_search)
  if (out == 1):
    rwins += 1
  if (out == 0):
    dwins += 1
  if (out == -1):
    bwins += 1
  out
winp.append(rwins)
winp.append(dwins)
winp.append(bwins)
print(winp)

KeyboardInterrupt: ignored

In [ ]:
# Your code here.
winp = []
rwins= 0
dwins=0
bwins=0
for i in range(10):
  conntect = ConnectFour(nrow=3, ncol=4, nwin=3)
  out = conntect.play_game(a_star_search, minmax_player)
  if (out == 1):
    rwins += 1
  if (out == 0):
    dwins += 1
  if (out == -1):
    bwins += 1
  out
winp.append(rwins)
winp.append(dwins)
winp.append(bwins)
print(winp)

Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?



A* vs Random:

A* wins most of the time. Ive ran it a few times out side of the 10 and usualy there is a win. I have swapped the two and Usually random loses to A* and its lookst to be aout 70-80% of the time from what I ahve ran. Usually when the black player blocks most of the in a row chips does the A* seem to have trouble.

The number of moves varie from 4-12 usually. Again it depends if the random player blocks the A* search player. However as the game get longer it seems that the random player will have a better chacne at winning. So the winning stratigy for A* seems to be to not get blocked. 

This makes me what to test a hurestic that counts the number of black and tries to stay away from it and build up rows. So add more value the futher away a row is from.


A* vs Min Max:

However in A* Vs Min MAx I realized something immportant: That A* doesnt check to see if its about to lose. As the huresitic I employed only priotizes getting things in a row. Therefore it always lose (and oddly enough same game every time because of the poor hurestic and min max taking advantage of it). 

Therefore, if I wanted to do better against Min Max I would try to look at opp win conditions and subtract it from my hurestic value and then bam I would see more draws as the proioty would become to stop losing and slight attempt at winning. 


Overall: A* requires a bit more of a flexable huresitc than what I applied. I applied one that priotized getting stacks of chips. However, while that works well againt a random player it doesnt work well against an AI that knows what they are doing. Therefore, it is also important to look at what the oppunent does (like what min max does with min and maxmizing players). 
